本文主要参考内容[An overview of gradient descent optimization algorithms](https://arxiv.org/pdf/1609.04747.pdf)以及https://ruder.io/optimizing-gradient-descent/index.html

梯度下降法是优化神经网络的最常用方法，在许多的深度学习框架中（比如：[tensorflow](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers),[pytorch](https://pytorch.org/docs/stable/optim.html)等)都实现了多种优化算法。但是，这些算法通常用作黑盒优化器，因为很难对它们的优缺点进行实用的解释。本文首先看一下梯度下降法的变种，然后说明模型在训练时的困难所在。接着介绍最常用的优化算法以及他们解决模型训练难点上起的作用。

梯度下降法的目地是最小化目标函数$J(\theta)$,通过对$\theta$沿着目标函数$J(\theta)$的梯度$\nabla_{\theta}J(\theta)$反方向来更新。学习率$\eta$决定更新步长大小。有关梯度下降法的更多说明[参见](https://cs231n.github.io/optimization-1/)

# 1、梯度下降法变种

有三种梯度下降法的变种，只是在计算目标函数梯度所用数据量不同的区别，数据量的大小可以可以影响梯度计算的准确度和参数更新的速度，数据越多，越准确，但是计算量会变大，会变慢，参有数更新也会变慢。

## 1.1 批量梯度下降法 batch gradient descent

是最初的梯度下降法，用训练集全部数据集来求梯度：$$\theta = \theta - \eta * \nabla_{\theta}J(\theta)$$ 由于用的数据太多，更新一次参数速度太慢，所以模型无法完成实时快速训练，但可以使在凸曲面上找到全局最优点，非凸曲面上的局部最优点。代码类似是：
```python
for i in range(n_epochs):
    params_grad = compute_grads(objective_function,data,params)
    params = params - lr*params_grad
```

## 1.2 随机梯度下降法 stochastic gradient descent

用单个样本$(x_i,y_i)$来求梯度并更新参数：$$\theta = \theta - \eta *\nabla_{\theta}J(\theta;x^{(i)};y^{(i)}) $$用单个样本来更新，速度快，但是波动大，收敛到最小值后，容易再次退出。估码类似：
```python
for i in range(nb_epochs):
  random.shuffle(data) #每个epoch都需要对数据进行打散
  for example in data:
    params_grad = compute_grads(objective_function, example, params)
    params = params - learning_rate * params_grad
```

## 1.3 小批量梯度下降法 mini-batch gradient descent

介于批量梯度下降法和随机梯度下降法之间，用训练数据中的部分数据来求梯度$$\theta = \theta - \eta *\nabla_{\theta}J(\theta;x^{(i:i+n)};y^{(i:i+n)}) $$ 好处理即快又稳定。代码类似：
```python
for i in range(nb_epochs):
  random.shuffle(data)
  for batch in get_batches(data, batch_size=50):
    params_grad = compute_grads(objective_function, batch, params)
    params = params - learning_rate * params_grad
```

# 2 、难点

- 很难选择合适的学习率，太小学习太慢，浪费时间； 太大无法收敛到最优处。
- 学习率周期，使学习率按照提前设定或损失域值来改变学习率，但是这样无法适应训练集的特性。
- 同一个学习率对所有参数进行更新也是不合适的。
- 陷入非凸函数的局部最小值或拐点。

# 3、常用梯度优化算法

## 3.1 Momentum

加入动量，记录之前梯度方向的一部分：$$v_t = \gamma*v_{t-1}+\eta * \nabla_{\theta}J(\theta)$$ $$\theta = \theta - v_t$$ $\gamma$通常设置为0.9或类似。以下用python实现该算法：

In [ ]:
class StochasticGradientDescent():
    def __init__(self, learning_rate=0.01, momentum=0):
        self.learning_rate = learning_rate 
        self.momentum = momentum
        self.w_updt = None

    def update(self, w, grad_wrt_w):
        # If not initialized
        if self.w_updt is None:
            self.w_updt = np.zeros(np.shape(w))
        # Use momentum if set
        self.w_updt = self.momentum * self.w_updt + (1 - self.momentum) * grad_wrt_w
        # Move against the gradient to minimize loss
        return w - self.learning_rate * self.w_updt
